# Amazon SageMaker Workshop
### _**Data Preparation**_

---
In this part of the workshop we will prepare the data to later train our churn model.

---

## Contents

1. [Background](#Background) - Getting the rawata prepared in the previous lab.
2. [Prepare](#Prepare) - Prepare the data with [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)
    * [Creating features](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-transform.html)
    * [Creating analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html)
    * [Analyzing the data and features](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html)
3. [Submiting the data to Feature Store](#FeatureStore) - Store the features created in [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/)
  
---

## Background

In the previous [Introduction lab](../0-Introduction/introduction.ipynb) we created a S3 bucket and uploaded the raw data to it.

Let's get started!

Get variables from previous configuration notebook:

In [7]:
%store -r bucket
%store -r region
%store -r prefix
%store -r s3uri_raw
%store -r docker_image_name
%store -r framework_version
bucket, prefix, s3uri_raw, region, docker_image_name, framework_version

('sagemaker-studio-us-east-1-924155096146',
 'xgboost-churn',
 's3://sagemaker-studio-us-east-1-924155096146/xgboost-churn/data/raw/churn.txt',
 'us-east-1',
 '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1',
 '1.3-1')

Let's import the libraries for this lab:

In [8]:
import sagemaker

sess = sagemaker.Session()
#bucket = sess.default_bucket()
#prefix = "sagemaker/DEMO-xgboost-churn"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
role

'arn:aws:iam::924155096146:role/service-role/AmazonSageMaker-ExecutionRole-20220711T131916'

In [9]:
import io
import os
import sys
import time
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display as dis
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from IPython import display 

# Importing Data on DataWrangler

To start, we will create a new flow and import the raw data to perform analysis and transformations on it. On the left menu, click on "Components and Registries", select "Data Wrangler" on the dropdown, and click "New Flow". This process is shown on the image below.

<img src="media/010-new_flow.png" width="30%" />

As soon as we click on "New Flow", Data Wrangler will be on a loading state. After a couple minutes you should be able to import the raw data on it. While we wait, we can rename our flow by right clicking on the flow tab and choosing "Rename Data Wrangler Flow...".

<img src="media/020-load_new_flow.png" width="100%" />

Let's call the file `churn.flow`

After Data Wrangler finishes loading, we may proceed importing our data. We'll be importing our data from Amazon S3. The following images guide us through the process.

<img src="media/030-importing_from_s3.png" width="100%" />

Using the search box or the table in the bottom, locate the S3 Bucket where our dataset is stored.

To remember the bucket and prefix run cell below:

In [10]:
s3uri_raw

's3://sagemaker-studio-us-east-1-924155096146/xgboost-churn/data/raw/churn.txt'

<img src="media/040-choosing_bucket.png" width="100%" />

Choose the "churn.csv" file, disable the "Enable sampling" checkbox, and click "Import". Feel free to check the data preview on the "Preview" session on the bottom!

<img src="media/050-choose_csv_preview.png" width="100%" />

# Data Analysis on Data Wrangler

Next, we are going to analyze our data by understanding feature distribution and how each of them impacts our target column. Data Wrangler allow us to perform such analysis inside user interface, let's start creating these analysis.

To start our analysis, we'll create a summary of our imported data. The summary can be rertieved by adding a new analysis on Data Wrangler. The following images show the step by step to create our table summary.

<img src="media/060-add_first_analysis.png" width="100%" />

<img src="media/070-table_summary.png" width="100%" />

As soon as you click on Preview or entering in your saved analysis, you'll see the table summary as shown in the image below.

<img src="media/080-table_summary_preview.png" width="100%" />

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Continuing our data analysis, we can leverage different Analysis type on Data Wrangler. Next, we can use Histogram to understand our feature distribution and how it impacts our target value. Going back to the "Analyze" tab, we can add a new Analysis. Then, we can choose Histogram as the "Analysis type" and select a feature to explore its distribution. On the following images we've chosen `Account Length` as feature and colored it by our target variable `Churn?`.

<img src="media/090-analyze_tab_create_new_analysis.png" width="100%" />

<img src="media/100-account_length_histogram.png" width="100%" />

As we can see, we are able to check the distribution of our feature and how its distribution relates to our target value. Feel free to create new histograms for any other feature!

For our last analysis, we'll leverage the "Quick Model" analysis provided by Data Wrangler. This analysis trains a Random Forest algorithm on its own and calculates a feature importance score for each feature on our dataset. You can learn more about the "Quick Model" analysis on this [page](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-analyses.html#data-wrangler-quick-model) of the Amazon SageMaker Data Wrangler documentation.


<img src="media/110-quick_model_analysis.png" width="100%" />

The higher the score, the more important the feature is. Therefore, feature `Day Mins` is the most important feature on our dataset according to the "Quick Model" analysis.

# Data Transforms on Data Wrangler

Go back to the DAG view by clicking on `Back to data flow` tab (on the top  left).

Follow the instructions on the image below:  
1 - Click on the plus (+) button  
2 - Click Edit data types  

<img src="media/120-edit_data_types.png" width="100%" />

3 - Find the column you want to change  
4 - Select the desired type from the dropdown (change **Area Code** to String)   
5 - Click preview  
6 - Click Add

<img src="media/130-area_code_to_object.png" width="100%" />

Once you finish, click "Back to data flow" on the top right corner

### Now lets drop the Phone column by adding a Transform  
1 - Click the plus (+)  
2 - Add Transform

<img src="media/140-add_transform.png" width="100%" />

3 - Click on manage columns  
4 - Select the `Phone` column from the dropdown (as shown in step 2)  
5 - Click on preview  
6 - Click add

<img src="media/150-steps_drop_phone_col.png" width="100%" />

### Now lets Drop a few more columns  
I'll be dropping the first (Day Charge) as an example, just repeat the steps in the image below for the following columns:
* "Day Charge"
* "Eve Charge"
* "Night Charge"
* "Intl Charge"

<img src="media/160-adding_new_transform.png" width="100%" />

<img src="media/170-drop_column_pt2.png" width="100%" />

### Now lets do OneHot Encoding using a custom transform
You can copy the code for the custom transform here:
```python
import pandas as pd

model_data = pd.get_dummies(df)

df = pd.concat(
  [model_data["Churn?_True."],
   model_data.drop( ["Churn?_False.", "Churn?_True."], axis=1)],
  axis=1
).rename( 
  columns = {
    "Churn?_True.": "Churn"
  }
)
```

<img src="media/180-custom_transform.png" width="100%" />

# Exporting Transformed data on Data Wrangler

After performing the transformations needed on our dataset, we'll export the transformed data to our S3 bucket. We are able to do so inside Data Wrangler UI by following the steps highlighted on the images below.

<img src="media/190-select_transforms_to_export.png" width="100%" />

<img src="media/200-exporting_to_s3.png" width="100%" />

After selecting the `Save to S3` option, a new notebook will be displayed similar to the one presented on the image below.

<img src="media/210-export_to_s3_notebook.png" width="100%" />

We can proceed to `Run all cells`, as descrribed in the notebook. The processing job will start and it will take a few minutes to finish. Upon completion, we'll see a similar output on the cell presented on the following image.

<img src="media/219-dw-notebook-run-all.png" width="100%" />

In the end of that notebook check that Processing Job is running (with the Data Wrangler Docker image):

<img src="media/220-processing_job_finished.png" width="100%" />

On the left menu, click on "Components and Registries", select "Experiments and trial" on the dropdown.

Select and double-click "Unassigned trial components":

<img src="media/225-open-experiments-processing.png" width="50%"/>

Select your processing job trial and open (right click and select "Open in trial details"):

<img src="media/225-find-processing-info.png" width="60%"/>

Go to the `Artifacts` tab, and **copy** the S3 URI of the output:

<img src="media/225-find-processing-output.png" width="100%" />

Paste your S3 URI below:

In [11]:
s3uri_processed = "s3://sagemaker-us-east-1-924155096146/export-flow-15-12-30-39-298879f7/output"

In [12]:
s3uri_processed_file = sagemaker.s3.S3Downloader.list(s3uri_processed)[0]
s3uri_processed_file

's3://sagemaker-us-east-1-924155096146/export-flow-15-12-30-39-298879f7/output/data-wrangler-flow-processing-15-12-30-39-298879f7/43a0e0e0-1052-400b-9a3f-0052ae0333ef/default/part-00000-33909da6-5870-47bb-a97b-e2b76e414caf-c000.csv'

If you want to check it in S3 console run the cell bellow and click the link:

In [13]:
from IPython.core.display import display, HTML
from sagemaker.s3 import parse_s3_url

out_bucket, out_prefix = parse_s3_url(s3uri_processed_file)
out_path = os.path.dirname(out_prefix)
out_file = os.path.basename(out_prefix)

s3_url_placeholder = "https://s3.console.aws.amazon.com/s3/buckets/{}?&prefix={}/"
display(HTML(f"<a href={s3_url_placeholder.format(out_bucket, out_path)}>Go to S3 console and check output of Data Wrangler</a>"))

In the S3 console you should see:

<img src="media/230-download_transformed_data_s3.png" width="100%" />

(If you want to download the data to you computer follow the steps in the image above)

Let's download the data to Studio:

In [14]:
sess.download_data(".", 
                   out_bucket, 
                   key_prefix=out_prefix)

Click the refresh button on Studio. You should see something like:

<img src="media/230-download_transformed_data_s3_local.png" width="50%" />

(The CSV file is downloaded)

In [15]:
out_file

'part-00000-33909da6-5870-47bb-a97b-e2b76e414caf-c000.csv'

In [16]:
model_data = pd.read_csv(out_file)
model_data.head()

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,128,25,265.1,110,197.4,99,244.7,91,10.0,...,0,0,0,0,1,0,1,0,0,1
1,0,107,26,161.6,123,195.5,103,254.4,103,13.7,...,0,0,0,0,1,0,1,0,0,1
2,0,137,0,243.4,114,121.2,110,162.6,104,12.2,...,0,0,0,0,1,0,1,0,1,0
3,0,84,0,299.4,71,61.9,88,196.9,89,6.6,...,0,0,0,1,0,0,0,1,1,0
4,0,75,0,166.7,113,148.3,122,186.9,121,10.1,...,0,0,0,0,1,0,0,1,1,0


Above we should see the transformed data with `Churn` in the first columns, the one-hot-encoded columns and so on.

Finally, let's break the data into **train, validation and test sets:**

In [17]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)
train_data.shape, validation_data.shape, test_data.shape

((2333, 70), (666, 70), (334, 70))

In [18]:
train_data.head(2)

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
1095,0,106,0,274.4,120,198.6,82,160.8,62,6.0,...,0,0,0,0,0,1,1,0,1,0
608,0,28,0,187.8,94,248.6,86,208.8,124,10.6,...,0,0,1,0,1,0,1,0,1,0


In [19]:
model_data.shape

(3333, 70)

Create CSV files for the 3 datasets:

In [20]:
train_file_name = "train.csv"
validation_file_name = "validation.csv"
test_file_name = "test.csv"

train_data.to_csv(train_file_name , header=False, index=False)
validation_data.to_csv(validation_file_name, header=False, index=False)
test_data.to_csv(test_file_name, header=False, index=False)

Lastly, we'll upload these files to S3.

In [21]:
# Return the URLs of the uploaded files, so they can be reviewed or used elsewhere
train_dir = f"{prefix}/data/train"
val_dir = f"{prefix}/data/validation"
test_dir = f"{prefix}/data/test"

s3uri_train = sagemaker.s3.S3Uploader.upload(train_file_name, f's3://{bucket}/{train_dir}')
s3uri_validation = sagemaker.s3.S3Uploader.upload(validation_file_name, f's3://{bucket}/{val_dir}')
s3uri_test = sagemaker.s3.S3Uploader.upload(test_file_name, f's3://{bucket}/{test_dir}')
s3uri_train, s3uri_validation, s3uri_test

('s3://sagemaker-studio-us-east-1-924155096146/xgboost-churn/data/train/train.csv',
 's3://sagemaker-studio-us-east-1-924155096146/xgboost-churn/data/validation/validation.csv',
 's3://sagemaker-studio-us-east-1-924155096146/xgboost-churn/data/test/test.csv')

Save the S3 URIs for the 3 datasets for later:

In [22]:
%store s3uri_train
%store s3uri_validation
%store s3uri_test

Stored 's3uri_train' (str)
Stored 's3uri_validation' (str)
Stored 's3uri_test' (str)
